# Prepare the Data

In [1]:
# Set up files using starter code
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

In [2]:
# Study data files with starter code
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

In [3]:
# Read the mouse data and study data with starter code
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

In [11]:
# Combine the data into a single dataframe
combo_df = pd.merge(study_results, mouse_metadata, how= 'inner', on='Mouse ID')

In [12]:
# Display the dataframe
combo_df.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,b128,5,45.651331,0,Capomulin,Female,9,22
2,b128,10,43.270852,0,Capomulin,Female,9,22
3,b128,15,43.784893,0,Capomulin,Female,9,22
4,b128,20,42.731552,0,Capomulin,Female,9,22


In [13]:
# check for the number of unique mice
unique_mice = combo_df["Mouse ID"].nunique()
unique_mice

249

In [19]:
duplicated_df = combo_df[combo_df.duplicated(subset=["Mouse ID","Timepoint"],keep= False)]
duplicated_df

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
860,g989,0,45.000000,0,Propriva,Female,21,26
861,g989,0,45.000000,0,Propriva,Female,21,26
862,g989,5,48.786801,0,Propriva,Female,21,26
863,g989,5,47.570392,0,Propriva,Female,21,26
864,g989,10,51.745156,0,Propriva,Female,21,26
865,g989,10,49.880528,0,Propriva,Female,21,26
866,g989,15,51.325852,1,Propriva,Female,21,26
867,g989,15,53.442020,0,Propriva,Female,21,26
868,g989,20,55.326122,1,Propriva,Female,21,26
869,g989,20,54.657650,1,Propriva,Female,21,26


In [20]:
unique_duplicates = duplicated_df["Mouse ID"].unique()
unique_duplicates

array(['g989'], dtype=object)

In [34]:
cleaned_df = combo_df.loc[combo_df["Mouse ID"] != 'g989']
cleaned_df.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,b128,5,45.651331,0,Capomulin,Female,9,22
2,b128,10,43.270852,0,Capomulin,Female,9,22
3,b128,15,43.784893,0,Capomulin,Female,9,22
4,b128,20,42.731552,0,Capomulin,Female,9,22


In [35]:
unique_cleaned = cleaned_df.nunique()
unique_cleaned

Mouse ID               248
Timepoint               10
Tumor Volume (mm3)    1633
Metastatic Sites         5
Drug Regimen            10
Sex                      2
Age_months              24
Weight (g)              16
dtype: int64

# Generate Summary Statistics

In [90]:
summary_drug_group = cleaned_df.groupby(["Drug Regimen"])
mean = summary_drug_group['Tumor Volume (mm3)'].mean()
med = summary_drug_group['Tumor Volume (mm3)'].median()
var = summary_drug_group['Tumor Volume (mm3)'].var()
std = summary_drug_group['Tumor Volume (mm3)'].std()
sem = summary_drug_group['Tumor Volume (mm3)'].sem()

volume_summary = pd.DataFrame({
                        "Mean Tumor Volume": mean,
                        "Median Tumor Volume" : med,
                        "Variance Tumor Volume" : var,
                        "Standard Deviation Tumor Volume" : std,
                        "SEM Tumor Volume" : sem
                        })

volume_summary

,Mean Tumor Volume,Median Tumor Volume,Variance Tumor Volume,Standard Deviation Tumor Volume,SEM Tumor Volume
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,52.320930,50.446266,43.852013,6.622085,0.544332
Ramicane,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,54.233149,52.431737,59.450562,7.710419,0.573111


In [91]:
summary_volume = summary_drug_group["Tumor Volume (mm3)"].agg(["mean","median","std","sem","var"])
summary_volume

,mean,median,std,sem,var
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,4.994774,0.329346,24.947764
Ceftamin,52.591172,51.776157,6.268188,0.469821,39.290177
Infubinol,52.884795,51.820584,6.567243,0.492236,43.128684
Ketapril,55.235638,53.698743,8.279709,0.603860,68.553577
Naftisol,54.331565,52.509285,8.134708,0.596466,66.173479
Placebo,54.033581,52.288934,7.821003,0.581331,61.168083
Propriva,52.320930,50.446266,6.622085,0.544332,43.852013
Ramicane,40.216745,40.673236,4.846308,0.320955,23.486704
Stelasyn,54.233149,52.431737,7.710419,0.573111,59.450562


# Create Pie Charts and Bar Charts

# Calculate quartiles, find outliers, and create a box plot

# Create a line plot and a scatter plot

# Calculate correlation and regression

# Submit your final analysis